In [2]:
import json
import boto3
zero_shot_prompts = [
"Input: 'Concerns about Saoirse Íngrid Björnsdóttir, with student ID 597213, SIN 213597846, and phone number 818-555-2134, have been raised due to his consistent failure to meet coursework deadlines. An academic probation letter is necessary.'",

]

payloads = []
for prompt in zero_shot_prompts:
    payloads.append(
        {
            "inputs": prompt, 
            "parameters": {"max_new_tokens": 100, "top_p": 0.9, "temperature": 0.6, "return_full_text": False},
        }
    )

endpoint_name = 'safegpt-model-endpoint'


def query_endpoint(payload):
    client = boto3.client("sagemaker-runtime")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, InferenceComponentName='meta-textgeneration-llama-2-7b-20240327-031345',
        ContentType="application/json",
        Body=json.dumps(payload),
    )
    response = response["Body"].read().decode("utf8")
    response = json.loads(response)
    return response
for payload in payloads:
    query_response = query_endpoint(payload)
    print(payload["inputs"])
    print(f"> {query_response[0]['generated_text']}")
    print("\n======\n")
    decoded_output = query_response[0]['generated_text']
    
    
import re
# Define a regex pattern to capture everything after "Expected Output: '"
# up to the closing quote
pattern = r"Expected Output: '(.*?)'"

# Search for the pattern in the output
match = re.search(pattern, decoded_output)

# Extract the matched part
if match:
    extracted_text = match.group(1)  # group(1) to get the content of the first capturing group
    print("Extracted Part:", extracted_text)
    
else:
    print("No match found")

Input: 'Concerns about Saoirse Íngrid Björnsdóttir, with student ID 597213, SIN 213597846, and phone number 818-555-2134, have been raised due to his consistent failure to meet coursework deadlines. An academic probation letter is necessary.'
>  ### Format: '<NAME>:Name, <STUDENT_ID>:ID, <SIN>:Number, <PHONE>:Phone Number' ### Expected Output: '<NAME>:Saoirse Íngrid Björnsdóttir, <STUDENT_ID>:597213, <SIN>:213597846, <PHONE>:818-555-2134' ### Task


Extracted Part: <NAME>:Saoirse Íngrid Björnsdóttir, <STUDENT_ID>:597213, <SIN>:213597846, <PHONE>:818-555-2134


In [5]:
user_text_list = extracted_text.split(',')
entity_dict = {}
print(user_text_list)
for item in user_text_list:
    item_list = item.split(":")
    print(item_list)
    entity_dict[item_list[0]] = item_list[1]
    

print(entity_dict)
    




['<NAME>:Saoirse Íngrid Björnsdóttir', ' <STUDENT_ID>:597213', ' <SIN>:213597846', ' <PHONE>:818-555-2134']
['<NAME>', 'Saoirse Íngrid Björnsdóttir']
[' <STUDENT_ID>', '597213']
[' <SIN>', '213597846']
[' <PHONE>', '818-555-2134']
{'<NAME>': 'Saoirse Íngrid Björnsdóttir', ' <STUDENT_ID>': '597213', ' <SIN>': '213597846', ' <PHONE>': '818-555-2134'}
